In [1]:
import numpy as np

In [2]:
def forw(a,b,pi,obs):
    T=len(obs)
    N=a.shape[0]
    alpha=np.zeros((N,T))
    alpha[:,0]=pi * b[:,obs[0]]
    for t in range(1,T):
        for j in range(N):
            alpha[j,t]=np.sum(alpha[:,t-1]* a[:,j]) * b[j,obs[t]]
    seq_prob= np.sum(alpha[:,-1])
    return seq_prob

In [3]:
def viterbi(a,b,pi,obs):
    T=len(obs)
    N=a.shape[0]
    gamma=np.zeros((N,T))
    path=np.zeros((N,T))
    gamma[:,0]=pi * b[:,obs[0]]
    for t in range(1,T):
        for j in range(N):
            (gamma[j,t],path[j,t])=max((gamma[i,t-1]* a[i,j] * b[j,obs[t]],i) for i in range(N))
    best_path=[]
    last_state= np.argmax(gamma[:,T-1])
    best_path.append(last_state)
    for t in range(T-1,0,-1):
        last_state= int(path[last_state, t])
        best_path.insert(0,last_state)
    
    seq_prob= np.max(gamma[:,-1])
    return best_path,seq_prob

In [4]:
def backw(a,b,pi,obs):
    T=len(obs)
    N=a.shape[0]
    beta=np.zeros((N,T))
    beta[:,-1]=1
    for t in range(T-2,-1,-1):
        for i in range(N):
            beta[i,t]= np.sum(a[i,:] *b [:,obs[t+1]] * beta[:,t+1])
            fin_prob=np.sum(pi* b[:,obs[0]] * beta[:,0])
    return fin_prob

In [5]:
A = np.array([[0.7,0.3],[0.5,0.5]])
B = np.array([[0.6,0.1,0.3],[0.1,0.7,0.2]]) 
pi = np.array([1,0])
obs_seq = [2, 1, 0]
states={'s1':0,'s2':1}

In [6]:
print("forward: ",forw(A,B,pi,obs_seq))

forward:  0.0315


In [7]:
print("Backward: ",backw(A,B,pi,obs_seq))

Backward:  0.03149999999999999


In [8]:
best,prob_v=viterbi(A,B,pi,obs_seq)
for i in range(len(best)):
    for key in states:
        if states[key]==best[i]:
            best[i]=key
print("Best: ",best)
print("verti prob",prob_v)

Best:  ['s1', 's2', 's1']
verti prob 0.0189


In [9]:
import pandas as pd
import numpy as np
import math as m
import random as r

In [10]:
dicts={"cola":0,"ice-t":1,"lem":2}
obs=[]
n=int(input("Enter the no of sequence: "))
for i in range(n):
    inp=input("Enter the sequence: ")
    obs.append(dicts[inp])
print(obs)
s=['A','B']
st=np.array([1,0])
tr=np.array([[0.7, 0.3],[0.5, 0.5]])
e=np.array([[0.6, 0.1, 0.3],[0.1, 0.7, 0.2]])
seq={0:"A",1:"B"}

Enter the no of sequence: 3
Enter the sequence: lem
Enter the sequence: ice-t
Enter the sequence: cola
[2, 1, 0]


In [11]:
def forward_p(obs,s,st,tr,e):
    nobs=len(obs)
    ns=len(s)
    
    f=np.zeros((ns,nobs))
    
    for i in range(ns):
        f[i,0]=st[i] * e[i,obs[0]]

    for t in range(1,nobs):
        for j in range(ns):
            f[j,t]=sum(f[i, t-1] * tr[i, j] * e[j, obs[t]] for i in range(ns))
    p=sum(f[i,nobs-1] for i in range(ns))
    return f,p


In [12]:
f,p=forward_p(obs,s,st,tr,e)
print("Forward: ",f)
print("prob: ",p) 

Forward:  [[0.3     0.021   0.02772]
 [0.      0.063   0.00378]]
prob:  0.0315


In [13]:
def viterbi_p(obs,s,st,tr,e):
    nobs=len(obs)
    ns=len(s)
    
    v=np.zeros((ns,nobs))
    path=np.zeros((ns,nobs))
    
    for i in range(ns):
        v[i,0]=st[i] * e[i,obs[0]]

    for t in range(1,nobs):
        for j in range(ns):
            (prob,state)=max((v[i, t-1] * tr[i, j] * e[j, obs[t]],i)for i in range(ns))
            v[j,t]=prob
            path[j,t]=state
    bp=[]
    ls=np.argmax(v[:,nobs-1])
    bp.append(ls)
    for h in range(nobs-1,0,-1):
        ls=int(path[ls,h])
        bp.insert(0,ls)
    pr=np.max(v[:,nobs-1])
    return bp,pr

In [14]:
bp,pr=viterbi_p(obs,s,st,tr,e)
print(bp)
print(pr)
ved=[]
for i in bp:
    ved.append(seq[i])
print(ved)   

[0, 1, 0]
0.0189
['A', 'B', 'A']


In [15]:
def backward_p(obs,s,st,tr,e):
    nobs=len(obs)
    ns=len(s)
    
    ba=np.zeros((ns,nobs))
    
    ba[:,-1]=1

    for t in range(nobs-2,-1,-1):
        for i in range(ns):
            ba[i,t]=sum(tr[i, j] * e[j, obs[t+1]] * ba[j, t+1] for j in range(ns))
    pb=sum(st[i] * e[i,obs[0]] * ba[i,0] for i in range(ns))
    return ba,pb

In [16]:
ba,pb=backward_p(obs,s,st,tr,e)
print("Backward: ",ba)
print("prob: ",pb) 

Backward:  [[0.105 0.45  1.   ]
 [0.145 0.35  1.   ]]
prob:  0.03149999999999999
